In [6]:
import requests
import pandas as pd
from tqdm import tqdm
from rouge_score import rouge_scorer

In [ ]:
# Few-shot 프롬프트 템플릿 (정답 스타일에 맞게 간결하게 수정)
FEW_SHOT_TEMPLATE = """대화를 한 문장으로 간결하게 요약하세요. 구체적인 숫자, 시간, 가격은 생략하고 핵심 행동만 작성합니다.

### 예시:

대화:
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요?
#Person2#: 건강검진을 받으려고 왔어요.
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요.
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요.
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 담배 피우세요?
#Person2#: 네.
#Person1#: 담배가 폐암하고 심장병의 주된 원인이에요. 끊으셔야 해요.
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요.
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다.
요약: Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다.

대화:
#Person1#: 저기요, 열쇠 세트 본 적 있어요?
#Person2#: 어떤 종류의 열쇠요?
#Person1#: 열쇠 다섯 개랑 작은 발 장식이 달려 있어요.
#Person2#: 아, 안타깝네요! 못 봤어요.
#Person1#: 그럼, 같이 좀 찾아주실 수 있어요?
#Person2#: 물론이죠. 도와드릴게요.
요약: #Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라고 요청합니다.

대화:
#Person1#: 안녕, 잭! 나 로즈야. 이번 주 토요일 저녁에 시간 있어?
#Person2#: 응, 왜?
#Person1#: 친구들이랑 저녁 모임 있는데 올래?
#Person2#: 좋아, 갈게.
요약: 로즈는 잭에게 전화하여 이번 주 토요일 저녁 식사에 초대한다.

대화:
#Person1#: 내일 면접인데 뭘 입어야 할지 모르겠어.
#Person2#: 정장에 넥타이 매는 게 좋아. 그리고 너무 긴장하지 말고 자신을 잘 표현해.
요약: #Person2#는 #Person1#에게 면접에서 정장과 넥타이를 착용하고 자신을 잘 표현하라고 조언합니다.

대화:
#Person1#: 이 코트 어때?
#Person2#: 색은 좋은데 사이즈가 작아 보여.
#Person1#: 더 큰 사이즈 있는지 물어볼까?
#Person2#: 응, 판매원한테 물어보자.
요약: 주와 조지가 코트를 고르며 더 큰 사이즈가 있는지 판매원에게 물어보려 한다.

대화:
#Person1#: 왜 사업을 시작하셨나요?
#Person2#: 지역 실업률 문제 때문이었어요. 관리 스타일은 엄격하기보다 기회를 많이 주는 편이에요.
#Person1#: 앞으로의 계획은요?
#Person2#: 새 부사장이 합류해서 저는 고객과 신제품에 집중하려고요. 개인적으로는 휴식 시간도 늘리고 싶어요.
요약: John은 지역 사람들과의 사업 시작 이유, 관리 스타일, 향후 사업 및 개인 생활 계획에 관해 설명합니다.

### 이제 다음 대화를 한 문장으로 간결하게 요약하세요:

대화:
{dialogue}
요약:"""

In [ ]:
def generate_summary(dialogue, model="exaone3.5:7.8b"):
    """EXAONE 모델을 사용하여 대화 요약 생성"""
    prompt = FEW_SHOT_TEMPLATE.format(dialogue=dialogue)
    
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "num_predict": 80,  # 더 짧게 제한
                    "stop": ["\n", "대화:", "###", "\n\n"]  # 한 문장 후 멈춤
                }
            },
            timeout=120
        )
        summary = response.json()["response"].strip()
        summary = post_process_summary(summary)
        return summary
    except Exception as e:
        print(f"Error: {e}")
        return ""


def post_process_summary(summary: str) -> str:
    """요약 후처리: 첫 문장만 추출, 길이 조절"""
    # 줄바꿈 제거
    summary = summary.replace("\n", " ").strip()
    
    # 불필요한 prefix 제거
    prefixes = ["요약:", "요약 :", "Summary:", "요약 :"]
    for prefix in prefixes:
        if summary.startswith(prefix):
            summary = summary[len(prefix):].strip()
    
    # 첫 문장만 추출 (마침표, 물음표, 느낌표 기준)
    for i, char in enumerate(summary):
        if char in ".!?" and i > 20:  # 최소 20자 이상일 때만
            # 다음 문자가 공백이거나 끝이면 문장 종료
            if i == len(summary) - 1 or summary[i+1] in " \n":
                summary = summary[:i+1]
                break
    
    return summary


def compute_rouge(pred, ref):
    """ROUGE 점수 계산"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    scores = scorer.score(ref, pred)
    return {
        'rouge1_f': scores['rouge1'].fmeasure,
        'rouge2_f': scores['rouge2'].fmeasure,
        'rougeL_f': scores['rougeL'].fmeasure,
        'total': scores['rouge1'].fmeasure + scores['rouge2'].fmeasure + scores['rougeL'].fmeasure
    }


def run_inference(df, model="exaone3.5:7.8b"):
    """데이터프레임에 대해 요약 생성"""
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        pred_summary = generate_summary(row['dialogue'], model)
        results.append({
            'fname': row.get('fname', idx),
            'dialogue': row['dialogue'],
            'pred_summary': pred_summary,
            'ground_truth': row.get('summary', '')
        })
    return pd.DataFrame(results)


def evaluate(results_df):
    """ROUGE 점수 평가"""
    scores = []
    for idx, row in results_df.iterrows():
        if row['ground_truth']:
            score = compute_rouge(row['pred_summary'], row['ground_truth'])
            scores.append(score)
    
    if scores:
        return {
            'rouge1_f': sum(s['rouge1_f'] for s in scores) / len(scores),
            'rouge2_f': sum(s['rouge2_f'] for s in scores) / len(scores),
            'rougeL_f': sum(s['rougeL_f'] for s in scores) / len(scores),
            'total': sum(s['total'] for s in scores) / len(scores)
        }
    return None

In [21]:
train_df = pd.read_csv("data/train.csv")  # 경로 수정
test_df = pd.read_csv("data/test.csv")    # 경로 수정

MODEL = "exaone3.5:7.8b"

print(f"Train: {len(train_df)}개")
print(f"Test: {len(test_df)}개")

Train: 12457개
Test: 499개


In [29]:
sample = train_df.iloc[0]
pred = generate_summary(sample['dialogue'], MODEL)

print("=== 생성 요약 ===")
print(pred)
print("\n=== 정답 요약 ===")
print(sample['summary'])

=== 생성 요약 ===
Dr. Hawkins 박사는 Mr. Smith 씨의 건강검진을 진행하며, 5년 동안 검진을 받지 않은 점을 지적하고 정기적인 검진의 중요성을 강조했습니다. 특히 흡연이 건강에 미치는 부정적 영향을 언급하며 금연을 권유했습니다. Mr. Smith 씨는 금연의 어려움을 호소했으나, Dr. Hawkins 박사는 금연을 돕는 프로그램과 약물에 대해 추가 정보를 제공하기로 했습니다.

=== 정답 요약 ===
Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다.


In [30]:
train_sample = train_df.sample(50, random_state=42)
train_results = run_inference(train_sample, MODEL)
scores = evaluate(train_results)

print(f"ROUGE-1: {scores['rouge1_f']:.4f}")
print(f"ROUGE-2: {scores['rouge2_f']:.4f}")
print(f"ROUGE-L: {scores['rougeL_f']:.4f}")
print(f"Total: {scores['total']:.4f}")

100%|██████████| 50/50 [01:09<00:00,  1.39s/it]

ROUGE-1: 0.5419
ROUGE-2: 0.3027
ROUGE-L: 0.4818
Total: 1.3263


In [24]:
# 결과 몇 개 직접 비교
for i in range(5):
    row = train_results.iloc[i]
    print(f"=== {i+1} ===")
    print(f"생성: {row['pred_summary']}")
    print(f"정답: {row['ground_truth']}")
    print(f"생성 길이: {len(row['pred_summary'].split())}")
    print(f"정답 길이: {len(row['ground_truth'].split())}")
    print()

=== 1 ===
생성: 로즈가 잭에게 토요일 저녁 6시에 친구들과의 모임에 초대할지 물어봤습니다.
정답: 로즈는 잭에게 전화하여 이번 주 토요일 저녁 식사에 초대한다.
생성 길이: 9
정답 길이: 9

=== 2 ===
생성: A는 B에게 면접 복장과 긴장 해소 방법을 조언받았습니다.
정답: #Person2#는 #Person1#에게 면접에서 정장과 넥타이를 착용하고 자신을 잘 표현하라고 조언합니다.
생성 길이: 8
정답 길이: 10

=== 3 ===
생성: John은 지역 실업률 문제로 외부 인력도 고용하며 사업을 시작했고, 엄격한 관리자보다는 기회를 많이 주는 스타일입니다. 향후에는 새 부사장 합류로 고객 및 신제품에 집중하고, 개인적으로는 매주 수요일 휴식을 통해 삶의 질을 향상시키려 합니다.
정답: John은 지역 사람들과의 사업 시작 이유, 관리 스타일, 향후 사업 및 개인 생활 계획에 관해 설명합니다.
생성 길이: 32
정답 길이: 16

=== 4 ===
생성: A는 #Person1#에게 베를린행 11시 40분 1등석 편도 티켓 가격과 게이트 4번을 안내했습니다. 티켓 구매는 기계에서 직접 해야 합니다.
정답: #Person1#는 베를린 행 비행기 시간을 묻고, #Person2#로부터 현재 이용 가능한 티켓과 가격 정보를 듣습니다. 티켓은 기계에서 구매할 수 있으며, 게이트 번호도 안내받습니다.
생성 길이: 18
정답 길이: 22

=== 5 ===
생성: 주는 친구에게 여러 코트를 보여주며 사이즈 문제로 고민하다가 판매원에게 도움을 청하기로 합니다.
정답: 주와 조지가 코트를 고르며 더 큰 사이즈가 있는지 판매원에게 물어보려 한다.
생성 길이: 12
정답 길이: 11



In [ ]:
# 전체 테스트 데이터에 대해 추론 및 제출 파일 생성
test_results = run_inference(test_df, MODEL)

# 제출 파일 저장
submission = test_results[['fname', 'pred_summary']].copy()
submission.columns = ['fname', 'summary']
submission.to_csv('submission_exaone.csv', index=False)
print("제출 파일 저장 완료: submission_exaone.csv")